# Train Model Job

In [125]:
from kfp.components import (
    create_component_from_func,
    InputPath,
    OutputPath
)
from typing import Dict

BASE_IMAGE = "quay.io/ibm/kubeflow-notebook-image-ppc64le:latest"


def train_model_job(
        dataset_directory: InputPath(str),
        train_specification: str,
        train_parameters: Dict[str, str],
        model_dir: OutputPath(str),
        train_mount: str = "/train",
        model_name: str = "my-model",
        base_image: str = "quay.io/ibm/kubeflow-notebook-image-ppc64le:latest",
        namespace: str = "",
        node_selector: str = "",
        pvc_name: str = "",
        pvc_size: str = "10Gi",
        cpus: str = "",
        gpus: int = 0,
        memory: str = "",
        remote_host: str = "",
        minio_url: str = "minio-service.kubeflow:9000",
        minio_secret: str = "mlpipeline-minio-artifact",
        upload_bucket: str = "jobdata"
):
    '''
    Trains a model. Once trained, the model is persisted to model_dir.

            Parameters:
                    dataset_directory: Path to the directory with training data.
                    train_specification: Training command as generated from a Python function using kfp.components.func_to_component_text.
                    train_parameters: Dictionary mapping formal to actual parameters for the training spacification.
                    model_dir: Target path where the model will be stored.
                    train_mount: Optional mounting point for training data of an existing PVC. Example: "/train".
                    model_name: Optional name of the model. Must be unique for the targeted namespace and conform Kubernetes naming conventions. Example: my-model.
                    base_image: Optional base image for model training. Example: quay.io/ibm/kubeflow-notebook-image-ppc64le:latest.
                    namespace: Optional namespace where the Job and associated volumes will be created. By default, the same namespace as where the pipeline is executed is chosen. Example: "user-example-com".
                    node_selector: Optional node selector for worker nodes. Example: nvidia.com/gpu.product: "Tesla-V100-SXM2-32GB".
                    pvc_name: Optional name to an existing persistent volume claim (pvc). If given, this pvc is mounted into the training job. Example: "music-genre-classification-j4ssf-training-pvc".
                    pvc_size: Optional size of the storage during model training. Storage is mounted into to the Job based on a persitent volume claim of the given size. Example: 10Gi.
                    cpus: Optional CPU limit for the job. Leave empty for cluster defaults (typically no limit). Example: "1000m".
                    gpus: Optional number of GPUs for the job. Example: 2.
                    memory: Optional memory limit for the job. Leave empty for cluster defaults (typically no limit). Example: "1Gi".
                    remote_host: Optional remote kubernetes cluster to run the job in. Requires to have a suitable bearer token mounted from a secret to this component such that the environment variable TOKEN can be used. Example: "https://XXX.XXX.XXX.XXX:443".
                    minio_url: Optional URL to MinIO object store. Username (MINIO_USER) and password (MINIO_PASS) have to be mounted via k8s_secret_key_to_env. Example: "minio-service.kubeflow:9000".
                    upload_bucket: Optional bucket name when using MinIO for data synchronization. Example: "temp".
    '''
    from datetime import datetime
    import errno
    import json
    import kfp
    from kubernetes import (
        client,
        config,
        utils,
        watch
    )
    import logging
    import os
    import shutil
    import sys
    import yaml

    logging.basicConfig(
        stream=sys.stdout,
        level=logging.INFO,
        format='%(levelname)s %(asctime)s: %(message)s'
    )
    logger = logging.getLogger()

    ###########################################################################
    # Helper Functions
    ###########################################################################
    def establish_kubernetes_connection():
        token = os.getenv('TOKEN')
        if (token == "" or remote_host == ""):
            logger.info("Token and remote_host not set. Using in-cluster configuration...")
            config.load_incluster_config()
            api_client = client.ApiClient()
            is_remote = False
            if(not os.path.exists(train_mount)):
                logger.warning(f"No local mount to {train_mount} found. Therefore, switching to remote data synchronization mode via MinIO. This will work but is slower compared to local mounts. Consider adding a mount to '{train_mount}' for this component by using a PVC inside your pipeline.")
                is_remote = True
        else:
            # see: https://github.com/kubernetes-client/python/blob/6d4587e18064288d031ed9bbf5ab5b8245460b3c/examples/remote_cluster.py
            logger.info("Token and remote_host found. Using remote cluster configuration...")
            configuration = client.Configuration()
            configuration.host = remote_host
            configuration.verify_ssl = False
            configuration.api_key = {"authorization": "Bearer " + token}
            api_client = client.ApiClient(configuration)
            is_remote = True

        return (api_client, is_remote)

    def clone_path(source, target):
        try:
            logger.info(f"Cloning source path {source} to {target} of training job...")
            shutil.copytree(source, target)
            logger.info("Cloning finished. Target path contents:")
            logger.info(os.listdir(target))
        except OSError as e:
            if e.errno in (errno.ENOTDIR, errno.EINVAL):
                shutil.copy(source, target)
            else:
                raise

    def sync_with_minio(
        inputs: dict,
        job_name: str,
        upload_bucket: str,
        minio_url: str,
        is_upload: bool,
        remove_minio_files: bool = False
    ):
        import json
        import logging
        from minio import Minio
        import os
        import sys
        import tarfile

        logging.basicConfig(
            stream=sys.stdout,
            level=logging.INFO,
            format='%(levelname)s %(asctime)s: %(message)s'
        )
        logger = logging.getLogger()

        def establish_minio_connection(minio_url):
            minio_user = os.getenv('MINIO_USER')
            minio_pass = os.getenv('MINIO_PASS')

            if (minio_user == "" or minio_pass == ""):
                err = "Environment variables MINIO_USER and MINIO_PASS need externally to be provided to this component using k8s_secret_key_to_env!"
                logger.error(err)
                raise Exception(err)

            return Minio(
                minio_url,
                access_key=minio_user,
                secret_key=minio_pass,
                secure=False
            )

        def path_to_tarfilename(pathname):
            return f"{pathname.replace(os.sep, '-')}.tar.gz"

        def make_tarfile(output_filename, source_dir):
            with tarfile.open(output_filename, "w:gz") as tar:
                tar.add(source_dir, arcname='.')

        def upload_to_minio(file, upload_bucket, job_name, minio_client):
            # Create models bucket if it does not yet exist
            response = minio_client.list_buckets()
            models_bucket_exists = False
            for bucket in response:
                if bucket.name == upload_bucket:
                    models_bucket_exists = True

            if not models_bucket_exists:
                minio_client.make_bucket(bucket_name=upload_bucket)

            minio_client.fput_object(
                bucket_name=upload_bucket,  # bucket name in Minio
                object_name=f"{job_name}/{file}",  # file name in bucket of Minio
                file_path=file,  # file path / name in local system
            )

        def download_from_minio(file, upload_bucket, job_name, minio_client, remove_minio_file):
            obj = f"{job_name}/{file}"
            data = minio_client.get_object(upload_bucket, obj)
            with open(file, 'wb') as file_data:
                for d in data.stream(32*1024):
                    file_data.write(d)
            if remove_minio_file:
                minio_client.remove_object(upload_bucket, obj)

        def extract_tarfile(tarfile_name, target):
            with tarfile.open(tarfile_name, "r:gz") as tar_gz_ref:
                tar_gz_ref.extractall(target)

        logger.info("Establishing MinIO connection...")
        minio_client = establish_minio_connection(minio_url)

        if (isinstance(inputs, str)):
            inputs = json.loads(inputs)

        if (isinstance(is_upload, str)):
            if (is_upload == "True"):
                is_upload = True
            else:
                is_upload = False

        for (source, target) in inputs:
            tarfilename = path_to_tarfilename(source)

            if (is_upload):
                logger.info(f"Tar.gz input {source} into {tarfilename}...")
                make_tarfile(tarfilename, source)

                logger.info(f'Uploading {tarfilename} to {upload_bucket}/{job_name}/{tarfilename}...')
                upload_to_minio(tarfilename, upload_bucket, job_name, minio_client)
            else:
                logger.info(f'Downloading {upload_bucket}/{job_name}/{tarfilename} to {tarfilename}...')
                download_from_minio(tarfilename, upload_bucket, job_name, minio_client, remove_minio_files)

                logger.info(f'Extracting {tarfilename} to {target}...')
                extract_tarfile(tarfilename, target)

                logger.info('Result:')
                logger.info(os.listdir(target))

    def generate_unique_job_name(model_name: str):
        epoch = datetime.today().strftime('%Y%m%d%H%M%S')
        return f"job-{model_name}-{epoch}"

    def initialize_namespace(namespace: str):
        SA_NAMESPACE = "/var/run/secrets/kubernetes.io/serviceaccount/namespace"

        if namespace == "":
            with open(SA_NAMESPACE) as f:
                namespace = f.read()
        namespace_spec = f"namespace: {namespace}"

        return (namespace, namespace_spec)

    def initialize_nodeselector(node_selector: str):
        if node_selector != "":
            node_selector = f"nodeSelector:\n        {node_selector}"
        return node_selector

    def initialize_init_container(
        is_remote: bool,
        base_image: str,
        inputs: Dict[str, str],
        job_name: str,
        upload_bucket: str,
        minio_url: str,
        minio_secret: str,
        mount_path: str
    ):
        if(not is_remote):
            return ""

        command_specification = kfp.components.func_to_component_text(
            func=sync_with_minio
        )

        # inner components loose type information as needed by lists/dicts
        # -> inputs need to be a string (using json here)
        inputs_json = json.dumps(inputs)
        parameters = {
            "inputs": inputs_json,
            "job_name": job_name,
            "upload_bucket": upload_bucket,
            "minio_url": minio_url,
            "is_upload": "False"
        }

        command, _, _ = initialize_command(
            command_specification,
            parameters
        )

        init_container = f"""initContainers:
          - name: init-inputs
            image: {base_image}
            command: {command}
            volumeMounts:
            - mountPath: {mount_path}
              name: training
            env:
            - name: MINIO_USER
              valueFrom:
                secretKeyRef:
                  name: {minio_secret}
                  key: accesskey
                  optional: false
            - name: MINIO_PASS
              valueFrom:
                secretKeyRef:
                  name: {minio_secret}
                  key: secretkey
                  optional: false
"""
        return init_container

    def initialize_command(
        specification: str,
        parameters: Dict[str, str],
        path_parameters: Dict[str, str] = {},
        mount_path: str = "/tmp"
    ):
        component_yaml = yaml.safe_load(specification)
        container_yaml = component_yaml["implementation"]["container"]
        command = container_yaml["command"]
        args = container_yaml["args"]

        actual_args = list()
        inputs = list()
        outputs = list()
        for idx, arg in enumerate(args):
            if type(arg) is dict:
                if "inputValue" in arg:
                    # required parameter (value)
                    key = arg["inputValue"]
                    if key in parameters:
                        actual_args.append(parameters[key])
                    else:
                        err = f"Required parameter '{key}' missing in component input!"
                        logger.error(err)
                        raise Exception(err)
                elif "if" in arg:
                    # optional parameter
                    key = arg["if"]["cond"]["isPresent"]
                    if key in parameters:
                        actual_args.append(f"--{key}")
                        actual_args.append(parameters[key])
                elif "inputPath" in arg:
                    # required InputPath
                    key = arg["inputPath"]
                    if key in parameters:
                        path_key = parameters[key]
                        if path_key in path_parameters:
                            mount = f"{mount_path}{path_parameters[path_key]}"
                            inputs.append((path_parameters[path_key], mount))
                            actual_args.append(mount)
                        else:
                            err = f"InputPath '{path_key}' unavailable in training component!"
                            logger.error(err)
                            raise Exception(err)
                    else:
                        err = f"Required parameter '{key}' missing in component input!"
                        logger.error(err)
                        raise Exception(err)
                elif "outputPath" in arg:
                    # required OutputPath
                    key = arg["outputPath"]
                    if key in parameters:
                        path_key = parameters[key]
                        if path_key in path_parameters:
                            mount = f"{mount_path}{path_parameters[path_key]}"
                            outputs.append((mount, path_parameters[path_key]))
                            actual_args.append(mount)
                        else:
                            err = f"OutputPath '{path_key}' unavailable in training component!"
                            logger.error(err)
                            raise Exception(err)
                    else:
                        err = f"Required parameter '{key}' missing in component input!"
                        logger.error(err)
                        raise Exception(err)
            else:
                # required parameter (key)
                actual_args.append(arg)

        command_with_initialized_args = json.dumps(command + actual_args)

        return command_with_initialized_args, inputs, outputs

    def initialize_fetch_command(
        outputs: Dict[str, str],
        job_name: str,
        upload_bucket: str,
        minio_url: str,
    ):
        command_specification = kfp.components.func_to_component_text(
            func=sync_with_minio
        )

        # inner components loose type information as needed by lists/dicts
        # -> outputs need to be a string (using json here)
        outputs_json = json.dumps(outputs)
        parameters = {
            "inputs": outputs_json,
            "job_name": job_name,
            "upload_bucket": upload_bucket,
            "minio_url": minio_url,
            "is_upload": "True"
        }
        command, _, _ = initialize_command(
            command_specification,
            parameters
        )
        return command

    def create_pvc_spec(pvc_name, namespace_spec, pvc_size):
        pvc_spec = f"""apiVersion: batch/v1
apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: {pvc_name}
  {namespace_spec}
spec:
  accessModes:
  - ReadWriteMany
  resources:
    requests:
      storage: {pvc_size}
"""
        return yaml.safe_load(pvc_spec)

    def create_minio_secret_spec(namespace_spec, minio_secret):
        minio_user = os.getenv('MINIO_USER')
        minio_pass = os.getenv('MINIO_PASS')
        minio_secret_spec = f"""apiVersion: v1
kind: Secret
metadata:
  name: {minio_secret}
  {namespace_spec}
stringData:
  accesskey: {minio_user}
  secretkey: {minio_pass}
"""
        return yaml.safe_load(minio_secret_spec)

    def create_train_job_spec(
        job_name,
        namespace_spec,
        node_selector,
        base_image,
        train_command,
        train_mount,
        cpus,
        memory,
        gpus,
        init_container,
        pvc_name
    ):
        if cpus:
            cpus = f"cpu: {cpus}"
        if memory:
            memory = f"memory: {memory}"
        if gpus:
            gpus = f"nvidia.com/gpu: {gpus}"

        job_spec = f"""apiVersion: batch/v1
kind: Job
metadata:
  name: {job_name}
  {namespace_spec}
spec:
  template:
    metadata:
      annotations:
        sidecar.istio.io/inject: "false"
    spec:
      {node_selector}
      containers:
        - name: training-container
          image: {base_image}
          command: {train_command}
          volumeMounts:
            - mountPath: {train_mount}
              name: training
          restartPolicy: Never
          resources:
            limits:
              {cpus}
              {memory}
              {gpus}
      {init_container}
      volumes:
        - name: training
          persistentVolumeClaim:
            claimName: {pvc_name}
      restartPolicy: Never
"""
        return yaml.safe_load(job_spec)

    def create_fetch_job_spec(
        job_name,
        namespace_spec,
        base_image,
        fetch_command,
        train_mount,
        minio_secret,
        pvc_name
    ):
        job_spec = f"""apiVersion: batch/v1
kind: Job
metadata:
  name: {job_name}
  {namespace_spec}
spec:
  template:
    metadata:
      annotations:
        sidecar.istio.io/inject: "false"
    spec:
      containers:
        - name: training-container
          image: {base_image}
          command: {fetch_command}
          volumeMounts:
            - mountPath: {train_mount}
              name: training
          restartPolicy: Never
          env:
          - name: MINIO_USER
            valueFrom:
              secretKeyRef:
                name: {minio_secret}
                key: accesskey
                optional: false
          - name: MINIO_PASS
            valueFrom:
              secretKeyRef:
                name: {minio_secret}
                key: secretkey
                optional: false
      volumes:
        - name: training
          persistentVolumeClaim:
            claimName: {pvc_name}
      restartPolicy: Never
"""
        return yaml.safe_load(job_spec)

    def submit_and_monitor_job(job_spec, namespace, api_client):
        objects = utils.create_from_yaml(
            api_client,
            yaml_objects=[job_spec]
        )
        job_name = objects[0][0].metadata.name

        logger.info("Waiting for job to succeed...")
        job_watch = watch.Watch()
        for job_event in job_watch.stream(
            batch_api.list_namespaced_job,
            namespace=namespace,
            label_selector=f"job-name={job_name}",
            timeout_seconds=0
        ):
            job = job_event['object']
            if job.status.active and not job.status.failed:
                logger.info("Monitoring pods of job...")

                # See https://stackoverflow.com/questions/65938572/kubernetes-python-client-equivalent-of-kubectl-wait-for-command
                pod_watch = watch.Watch()
                for pod_event in pod_watch.stream(func=core_api.list_namespaced_pod,
                                          namespace=namespace,
                                          label_selector="controller-uid=" + job.metadata.labels["controller-uid"],
                                          timeout_seconds=0):
                    pod = pod_event["object"]
                    pod_name = pod.metadata.name

                    logger.info(f"Pod {pod_name} status: {pod.status.phase}")
                    if pod.status.phase == "Running" or pod.status.phase == "Succeeded" or pod.status.phase == "Failed":
                        logger.info("==============================================================================")
                        logger.info("==============================================================================")
                        logger.info(f"=== Streaming logs of pod {pod_name}...")
                        logger.info("==============================================================================")
                        logger.info("==============================================================================")

                        log_watch = watch.Watch()
                        for log_event in log_watch.stream(
                            core_api.read_namespaced_pod_log,
                            name=pod_name,
                            namespace=namespace,
                            follow=True,
                            _return_http_data_only=True,
                            _preload_content=False
                        ):
                            print(log_event)
                        logger.info("==============================================================================")
                        logger.info("==============================================================================")

                        pod_watch.stop()

                        if pod.status.phase == "Failed":
                            err = "Job failed while executing."
                            logger.error(err)
                            raise Exception(err)
                        break
                    if pod_event["type"] == "DELETED":
                        err = "Pod was deleted while we where waiting for it to start."
                        logger.error(err)
                        raise Exception(err)
            if job.status.succeeded:
                job_watch.stop()
                logger.info("Job finished.")
                break

            if not job.status.active and job.status.failed:
                job_watch.stop()
                logger.error("Job failed!")
                raise Exception("Job failed!")

    ###########################################################################
    # Main Workflow
    ###########################################################################

    logger.info("Establishing cluster connection...")
    api_client, is_remote = establish_kubernetes_connection()
    batch_api = client.BatchV1Api(api_client)
    core_api = client.CoreV1Api(api_client)

    logger.info("Initializing configurations...")
    job_name = generate_unique_job_name(model_name)
    if is_remote:
        minio_secret = f"{job_name}-minio-secret"
    namespace, namespace_spec = initialize_namespace(namespace)
    pvc_name = f"{job_name}-pvc"
    node_selector = initialize_nodeselector(node_selector)

    path_parameters = {
            "dataset_directory": dataset_directory,
            "model_dir": model_dir
        }
    train_command, inputs, outputs = initialize_command(
        train_specification,
        train_parameters,
        path_parameters,
        train_mount
    )
    init_container = initialize_init_container(
        is_remote,
        base_image,
        inputs,
        job_name,
        upload_bucket,
        minio_url,
        minio_secret,
        train_mount
    )

    logger.info("=======================================")
    logger.info("Derived configurations")
    logger.info("=======================================")
    logger.info(f"job_name: {job_name}")
    logger.info(f"namespace: {namespace}")
    logger.info(f"is_remote: {is_remote}")
    logger.info(f"minio_url: {minio_url}")
    logger.info(f"minio_secret: {minio_secret}")
    logger.info("inputs (input paths send to job):")
    for source, target in inputs:
        logger.info(f"- {source} -> {upload_bucket}/{job_name}/{target}")
    logger.info("outputs (output paths returning from job):")
    for source, target in outputs:
        logger.info(f"- {target} <- {upload_bucket}/{job_name}/{source}")
    logger.info(f"train_command: {train_command}")
    logger.info("=======================================")

    if (is_remote):
        logger.info("Using MinIO to sync data with a new remote PVC for the job...")
        sync_with_minio(inputs, job_name, upload_bucket, minio_url, is_upload=True)
        pvc_spec = create_pvc_spec(pvc_name, namespace_spec, pvc_size)
        minio_secrets_spec = create_minio_secret_spec(namespace_spec, minio_secret)
        utils.create_from_yaml(
            api_client,
            yaml_objects=[pvc_spec, minio_secrets_spec]
        )
    else:
        logger.info(f"Pushing inputs to local {train_mount} mount as shared with job environment...")
        for (source, target) in inputs:
            clone_path(source, target)

    logger.info("Creating train job specification...")
    train_job_spec = create_train_job_spec(
        job_name,
        namespace_spec,
        node_selector,
        base_image,
        train_command,
        train_mount,
        cpus,
        memory,
        gpus,
        init_container,
        pvc_name
    )

    logger.info(f"Starting train job '{namespace}.{job_name}'...")
    submit_and_monitor_job(train_job_spec, namespace, api_client)

    logger.info("Receiving training outputs...")
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    if (is_remote):
        fetch_command = initialize_fetch_command(
            outputs,
            job_name,
            upload_bucket,
            minio_url
        )
        fetch_job_name = f"{job_name}-fetch"

        logger.info("Creating fetch job specification...")
        fetch_job_spec = create_fetch_job_spec(
            fetch_job_name,
            namespace_spec,
            base_image,
            fetch_command,
            train_mount,
            minio_secret,
            pvc_name
        )

        logger.info(f"Starting fetch job '{namespace}.{fetch_job_name}'...")
        submit_and_monitor_job(fetch_job_spec, namespace, api_client)

        logger.info("Fetching output data from MinIO & deleting it afterwards...")
        sync_with_minio(outputs, job_name, upload_bucket, minio_url, is_upload=False, remove_minio_files=True)

        logger.info(f"Deleting Job {fetch_job_name}...")
        batch_api.delete_namespaced_job(fetch_job_name, namespace)
    else:
        logger.info(f"Fetching outputs to local {train_mount} mount as shared with job environment...")
        for (source, target) in outputs:
            clone_path(source, target)

    logger.info(f"Deleting Job {job_name}...")
    batch_api.delete_namespaced_job(job_name, namespace)
    if (is_remote):
        logger.info(f"Deleting remote PVC {pvc_name}...")
        core_api.delete_namespaced_persistent_volume_claim(pvc_name, namespace)

        logger.info(f"Deleting remote MinIO secret {minio_secret}...")
        core_api.delete_namespaced_secret(minio_secret, namespace)

    logger.info("Finished.")


train_model_job_comp = create_component_from_func(
    func=train_model_job,
    output_component_file='component.yaml',
    base_image=BASE_IMAGE
)